# This file currently exist to test the BGG API. The code is not yet in any particular order

Eventual information I hope to obtain:
 * Take about 100 comments from each to find what people do or don't like about certain games
   * See if that has any relation to the game's categories or mechanism
   * (Not sure if possible) See how these people have rated other games, find similar tastes
 * Take the score of a game and see how it relates to player count, age, and mechanism
   * Are some mechanisms consistently more popular? Are some more divisive?
   * Are newer games rated more highly? Have games gotten better?

# Setup

In [ ]:
import requests
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import sqlite3

In [ ]:
con = sqlite3.connect('bgg.db')
cur = con.cursor()

In [ ]:
ERROR_MESSAGE = error_message = ['\n\tYour request for this collection has been accepted and will be processed.  Please try again later for access.\n']

In [ ]:
def get_player_count_ratings(game_id):
    params = {'id' : game_id}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    s = BeautifulSoup(response.text)
    player_votings = s.find_all('results')
    player_counter = 0

    # Lookup if a rating refers to best, rec or not rec
    rec_li = ['Best', 'Recommended', 'Not Recommended']


    # output used to make SQL table
    full_rating_li = []

    for val in player_votings:
        try: 
            for idx, elem in enumerate(val.find_all('result')):
                full_rating_li.append((game_id, val['numplayers'], rec_li[idx], elem['numvotes']))
        except KeyError:
            pass

    return full_rating_li 

In [ ]:
def get_user_ratings(username):
    error_message = ['\n\tYour request for this collection has been accepted and will be processed.  Please try again later for access.\n']
    
    got_data = False
    # Make API call to get XML data
    while not got_data:
        params = {'username' : username, 'type': 'boardgame', 'rated' : '1', 'brief' : '1', 'stats' : '1'}
        response = requests.get('https://www.boardgamegeek.com/xmlapi2/collection?', params)
        s = BeautifulSoup(response.text, "xml")
        
        try:
            s.find('message').contents
        except AttributeError:
            got_data = True
        finally:
            time.sleep(1)
    
    # Initialize dataframe
    rating_df = pd.DataFrame(columns = ['id', 'rating'])

    # Loop through and output ratings to a dataframe
    for i in s.find_all('item'):
        game_id = i['objectid']
        
        # Make a soup out of the item info to extract rating
        inner_soup = BeautifulSoup(str(i))
        rv = inner_soup.find('stats')
        inner_soup2 = BeautifulSoup(str(rv))
        rating = inner_soup2.find('rating')
        rating = float(rating['value'])
        
        # Append rating to df
        temp_df = pd.DataFrame([[game_id,rating]], columns = ['id', 'rating'])
        rating_df = rating_df.append(temp_df)
    
    rating_df['index'] = username
    rating_df.rename({'index' :'username'}, inplace=True)
    
    return rating_df

In [ ]:
con = sqlite3.connect('bgg.db')
cur = con.cursor()

# Get Player Counts

In [ ]:
# Get list of top 1000 games from SQL table
game_li = []

results=cur.execute("select id FROM TOP_BGG_GAMES")
results=results.fetchall()

for num in results:
    game_li.append(num[0])

In [ ]:
# For each game, get the min and max player count:
min_max_player_li = []

i = 0
while i < len(game_li):
    game_id = game_li[i]
    params = {'id' : game_id,'stats' :'1'}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    soup = BeautifulSoup(response.text, "xml")

    try: 
        min_play = soup.find('minplayers')['value']
        max_play = soup.find('maxplayers')['value']
        i += 1
    except TypeError:
        # If it fails, we need to wait so we can use the API gain
        time.sleep(15)
    
    min_max_player_li.append([game_id, min_play, max_play])

In [ ]:
# Create table
cur.execute(''' CREATE TABLE BGG_PLAYER_COUNTS
               (id text, min_players text, max_players text)''')

In [ ]:
# Remove duplicates from API
player_li_df = pd.DataFrame(min_max_player_li, columns = ['GameId','MinPlayers','MaxPlayers'])
player_li_df.drop_duplicates(subset = 'GameId', keep='first', inplace=True)
player_li_df.reset_index(inplace=True)
player_li_df.shape

In [ ]:
# Convert dataframe to tuple
df = player_li_df
tuple_li = []
for i in range(len(df)):
    tuple_ = str(df.loc[i, 'GameId']), df.loc[i, 'MinPlayers'], str(df.loc[i, 'MaxPlayers'])
    tuple_li.append(tuple_)

In [ ]:
# Enter tuple into SQL table
cur.executemany("insert into BGG_PLAYER_COUNTS values (?, ?, ?)", tuple_li)

In [ ]:
con.commit()

# Obtain game categories and mechanism

In [ ]:
category_li, mechanic_li = [],[]
loop_len = len(game_li)

i = 0
while i < loop_len:
    game_id = game_li[i]
    params = {'id' : game_id}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    soup = BeautifulSoup(response.text, 'xml')
    info_ = soup.find_all('link')
    
    try:
        for x in info_:
            if x['type'] == 'boardgamecategory':
                category_li.append((game_id, x['value']))
            elif x['type'] == 'boardgamemechanic':
                mechanic_li.append((game_id, x['value']))
        i += 1 # increment the loop
    except TypeError:  # occurs when API times out
        time.sleep(10) # wait ten seconds so we can run the API gain

In [ ]:
# Create table
cur.execute(''' CREATE TABLE BGG_CATEGORIES
               (id text, category text);
               ''')

cur.execute('''CREATE TABLE BGG_MECHANICS
               (id text, mechanic text);''')

In [ ]:
# Enter tuple into SQL table
cur.executemany("insert into BGG_CATEGORIES values (?, ?)", category_li)
cur.executemany("insert into BGG_MECHANICS values (?, ?)", mechanic_li)

In [ ]:
con.commit()

# Key Game Stats

In [ ]:
def get_game_stats(game):
    params = {'id' : game,'stats' :'1'}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    s = BeautifulSoup(response.text, "xml")
    all_stats = s.find('ratings')
    avg_rating = all_stats.find('average')['value']
    avg_bayes_rating = all_stats.find('bayesaverage')['value']
    std = all_stats.find('stddev')['value']
    ratings = all_stats.find('usersrated')['value']
    weight = all_stats.find('averageweight')['value']
    
    return((game, avg_rating, avg_bayes_rating, std, ratings, weight))

In [ ]:
game_rating_li = []
for idx, game in enumerate(game_li):
    got_data = False
    
    while not got_data:
        try:
            temp = get_game_stats(game)
            got_data = True
        except AttributeError:
            time.sleep(10)
    # print(idx) 
    game_rating_li.append(temp)

In [ ]:
cur.execute('''CREATE TABLE BGG_GAME_STATS
               (id text, avg_rating text, avg_bayes_rating text, std text, ratings text, weight text);''')

In [ ]:
cur.executemany("insert into BGG_GAME_STATS values (?, ?, ?, ?, ?, ?)", game_rating_li)

In [ ]:
con.commit()

# Num of Players Voting

In [ ]:
full_player_rating_li = []
for g in game_li:
    current_li = get_player_count_ratings(g)
    if current_li == []:
        time.sleep(10)
        current_li = get_player_count_ratings(g)
    full_player_rating_li.extend(current_li)

In [ ]:
cur.execute(''' CREATE TABLE BGG_PLAYER_RATINGS
               (id text, player_count text, player_recommendation text, votes text);
               ''')

In [ ]:
cur.executemany("insert into BGG_PLAYER_RATINGS values (?, ?, ?, ?)", full_player_rating_li)

In [ ]:
con.commit()

# Join Tables

In [ ]:
cur.execute('''CREATE TABLE TOP_GAMES_FULL
           
           AS
           
        SELECT t1.id, title, year, t2.min_players, t2.max_players
           ,avg_rating, avg_bayes_rating, std, ratings, weight
        FROM TOP_BGG_GAMES t1
            LEFT JOIN BGG_PLAYER_COUNTS t2
                ON t1.id = t2.id
            LEFT JOIN  BGG_GAME_STATS t3
                ON t1.id = t3.id
           
           ''')

In [ ]:
con.commit()

In [ ]:
results = cur.execute('''
        SELECT t1.id, title, max_players
           ,avg_rating, std
        FROM TOP_GAMES_FULL t1
           
           ''')

In [ ]:
result_li = [result for result in results]

In [ ]:
df = pd.DataFrame(data=result_li, columns=['id', 'title', 'max_players', 'score', 'std_score'])

In [ ]:
# convert data types
cols = ['max_players', 'score', 'std_score']
for col in cols:
    df[col] = df[col].astype('float')
df.dtypes

In [ ]:
# solo games 
df[df.max_players == 1]

In [ ]:
# duel games
df[df.max_players == 2]

In [ ]:
# Large Group games
df[df.max_players > 5]

In [ ]:
#What's the largest?
print(df.max_players.max())

# Which games?
df[df['max_players'] == 100]

In [ ]:
# What's the most divisive game?
df[df['std_score']==df.std_score.max()]

In [ ]:
# The least divisive?
df[df['std_score']==df.std_score.min()]

In [ ]:
params = {'id' : 13,'stats' :'1'}
response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
s = BeautifulSoup(response.text, "xml")
s.find('playingtime')

# Coda: Adding Play time

Initially, I felt that the previous table was complete. I decided to add playtime as well. I should have added it before, though this gives me the chance to test the Alter table functions

In [83]:
i = 0
playtime_li = []
while i < len(game_li):
    game_id = game_li[i]
    params = {'id' : game_id,'stats' :'1'}
    try:
        response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
        s = BeautifulSoup(response.text, "xml")
        play_time = s.find('playingtime')['value']
        playtime_li.append((game_id, play_time))
        i += 1
    except TypeError:
        time.sleep(20)

In [85]:
cur.execute(''' CREATE TABLE BGG_PLAY_TIME
               (id text, play_time text);
               ''')

cur.executemany("insert into BGG_PLAY_TIME values (?, ?)", playtime_li)

In [87]:
cur.execute('''  ALTER TABLE TOP_GAMES_FULL
     ADD COLUMNS play_time text ''')

In [91]:
cur.execute(''' DELETE  FROM TOP_GAMES_FULL  WHERE TRUE''')

In [94]:
cur.execute('''INSERT INTO TOP_GAMES_FULL
           
        SELECT t1.id, title, year, t2.min_players, t2.max_players
           ,avg_rating, avg_bayes_rating, std, ratings, weight, t4.play_time
        FROM TOP_BGG_GAMES t1
            LEFT JOIN BGG_PLAYER_COUNTS t2
                ON t1.id = t2.id
            LEFT JOIN  BGG_GAME_STATS t3
                ON t1.id = t3.id
            LEFT JOIN BGG_PLAY_TIME t4
                ON t1.id = t4.id''')

In [107]:
results = cur.execute('PRAGMA table_info(TOP_GAMES_FULL);')
results = results.fetchall()
result_li = [r for r in results]
result_li

[(0, 'id', 'TEXT', 0, None, 0),
 (1, 'title', 'TEXT', 0, None, 0),
 (2, 'year', 'TEXT', 0, None, 0),
 (3, 'min_players', 'TEXT', 0, None, 0),
 (4, 'max_players', 'TEXT', 0, None, 0),
 (5, 'avg_rating', 'TEXT', 0, None, 0),
 (6, 'avg_bayes_rating', 'TEXT', 0, None, 0),
 (7, 'std', 'TEXT', 0, None, 0),
 (8, 'ratings', 'TEXT', 0, None, 0),
 (9, 'weight', 'TEXT', 0, None, 0),
 (10, 'play_time', 'play_time text', 0, None, 0)]

In [106]:
cur.execute('ALTER TABLE TOP_GAMES_FULL RENAME COLUMN COLUMNS TO play_time;')

In [108]:
con.commit()

Playtime added!

# Scratch Code

In [ ]:
# Find the votes with respect to the number of players
results = soup.find_all('results', {'numplayers':'1'})

# general text for vote counts
soup2 = BeautifulSoup(str(results[0]), 'xml')
print(soup2)

# Specific value of the recommendation for 1 player
# Side note, this is a troll vote as Catan requires at least 3 player
soup2.find('result', {'value' :'Best'})['numvotes']

In [ ]:
# We look at certain users from the comments and see what other games they might like

params = {'name' : 'agentzen', 'hot' : '1', 'top' : '1', 'domain' : 'boardgame',}
response = requests.get('https://www.boardgamegeek.com/xmlapi2/user?', params)
s = BeautifulSoup(response.text, "xml")
s

In [ ]:
# We look at certain users from the comments and see what other games they might like
params = {'username' : 'agentzen', 'type': 'boardgame', 'rated' : '1', 'brief' : '1', 'stats' : '1'}
response = requests.get('https://www.boardgamegeek.com/xmlapi2/collection?', params)
s = BeautifulSoup(response.text, "xml")
s

In [ ]:
# Comments
comment_li = []

# Obtaining ten pages of user comments/ratings
for i in range(1,11):
    params = {'id' : '13', 'ratingcomments' : '1', 'comments' : '1', 'pagesize' : '100', 'page':[3,4], 'stats' :str(i)}
    response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?', params)
    comments = soup.find_all("comment")
    for com in comments:
        comment_li.append(com)

In [ ]:
test_df = get_user_ratings('agent_t_bib')
test_df